In [65]:
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt 
import glob
import os
import xarray as xr 
import pandas as pd
import numpy as np


cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=42,                   # 24 cores per node
    processes=10,                # One process per task
    memory="400GB",             # --mem
    walltime="01:15:00",        # --time
    queue="med",                # --partition
    log_directory=".",          # Logs will be saved to the current directory
)

"""
cluster = SLURMCluster(
    job_name="Climt1",
    cores=24,                        # 24 CPU cores per GPU
    processes=1,                     # One Dask worker per job, using one GPU
    memory="100GB",                  # Allocate ~1/4 of node memory if total is ~400GB
    walltime="01:15:00",
    queue="gpu1",
    log_directory=".",
    job_extra_directives=[
        "--gres=gpu:2",              # Request 1 GPU
        "--ntasks=1",                # One task per job
        "--cpus-per-task=24"        # Match cores per task
    ],)
"""

from dask.distributed import Client


cluster.scale(jobs=1)
#cluster.adapt(minimum=1,maximum=4)
client = Client(cluster)
client

/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33157 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:33157/status,
Dashboard: http://10.42.239.61:33157/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:41681,Workers: 0
Dashboard: http://10.42.239.61:33157/status,Total threads: 0
Started: Just now,Total memory: 0 B


Task exception was never retrieved
future: <Task finished name='Task-1297325' coro=<Client._gather.<locals>.wait() done, defined at /home1/nalex2023/.local/lib/python3.10/site-packages/distributed/client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/client.py", line 2386, in wait
    raise AllExit()
distributed.client.AllExit


In [64]:
client.close()
client.shutdown()

In [66]:

filerh = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/Rh_mid/'

fileu = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/u_comp/'

filev = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/v_comp/'

file_sst_sp = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/surfacePT/'


In [67]:

def prepro(ds):
    ds_mean = ds.mean(dim='level')
    return ds_mean


dset_rh = xr.open_mfdataset(filerh+'*.nc',chunks={'time':1000,'lat':50,'lon':50},preprocess=prepro,parallel=True).sel(expver=1)#.compute()


In [13]:
save_fol = '/home1/nalex2023/Datasets/trop_OUTS/'

#dset_rh.to_netcdf(f'{save_fol}RH_MID_2000_2023_ERA5.nc')


In [49]:
dset_rh

<xarray.Dataset> Size: 14GB
Dimensions:    (time: 34667, latitude: 165, longitude: 313)
Coordinates:
  * longitude  (longitude) float32 1kB -83.0 -82.75 -82.5 ... -5.5 -5.25 -5.0
  * latitude   (latitude) float32 660B 43.0 42.75 42.5 42.25 ... 2.5 2.25 2.0
  * time       (time) datetime64[ns] 277kB 2000-06-01 ... 2023-10-03T06:00:00
    expver     int32 4B 1
Data variables:
    r          (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>

In [68]:
dset_rh.compute()

CommClosedError: in <TCP (closed) ConnectionPool.gather local=tcp://10.42.239.61:52350 remote=tcp://10.42.239.61:41681>: ConnectionResetError: [Errno 104] Connection reset by peer

In [28]:


def preprocess(uvdata):
    data = uvdata   
    meanup = data.sel(level=[200,250,300]).mean(dim='level')
    meanlow = data.sel(level=[775,800,925]).mean(dim='level')
    
    shear_c = np.abs(meanup - meanlow)
    
    return shear_c



dset_u = xr.open_mfdataset(fileu+'*.nc',chunks={'time':1000},preprocess=preprocess,parallel=True).sel(expver=1)
dset_v = xr.open_mfdataset(filev+'*.nc',chunks={'time':1000},preprocess=preprocess,parallel=True).sel(expver=1)



dset_shear = np.sqrt(dset_u['u']**2 + dset_v['v']**2)

dset_shear = dset_shear.to_dataset(name='shear')

dset_shear

<xarray.Dataset> Size: 32GB
Dimensions:    (longitude: 341, latitude: 193, time: 60826)
Coordinates:
  * longitude  (longitude) float32 1kB -83.0 -82.75 -82.5 ... 1.5 1.75 2.0
  * latitude   (latitude) float32 772B 43.0 42.75 42.5 42.25 ... -4.5 -4.75 -5.0
  * time       (time) datetime64[ns] 487kB 2000-01-01 ... 2023-10-26T03:00:00
    expver     int32 4B 1
Data variables:
    shear      (time, latitude, longitude) float64 32GB dask.array<chunksize=(966, 193, 341), meta=np.ndarray>

In [ ]:
cluster.close()


AttributeError: 'SLURMCluster' object has no attribute 'shutdown'

In [29]:
dset_shear.to_netcdf(f'{save_fol}SHEAR_2000_2023.nc')

OSError: [Errno -101] NetCDF: HDF error: '/home1/nalex2023/Datasets/trop_OUTS/SHEAR_2000_2023.nc'

In [14]:
new_rh = xr.open_dataset(f'{save_fol}RH_MID_2000_2023_ERA5.nc',chunks={'time':1000}).sel(time=slice('2000','2022'))

new_shear = xr.open_dataset(f'{save_fol}SHEAR_2000_2023_ERA5.nc',chunks={'time':1000}).sel(time=slice('2000','2022'))

st_sst = xr.open_mfdataset(file_sst_sp+'*.nc',chunks={'time':1000},parallel=True).sel(expver=1)#.sel(time=slice('2000','2022'))

In [15]:
final_frame = new_rh.copy()

final_frame['shear'] = new_shear['shear'].reindex(time=new_rh['time'], method='nearest')
final_frame['sst'] = st_sst['sst'].reindex(time=new_rh['time'], method='nearest')
final_frame['sp'] = st_sst['sp'].reindex(time=new_rh['time'], method='nearest')

In [17]:
final_frame

<xarray.Dataset> Size: 56GB
Dimensions:    (longitude: 313, latitude: 165, time: 33672)
Coordinates:
  * longitude  (longitude) float32 1kB -83.0 -82.75 -82.5 ... -5.5 -5.25 -5.0
  * latitude   (latitude) float32 660B 43.0 42.75 42.5 42.25 ... 2.5 2.25 2.0
  * time       (time) datetime64[ns] 269kB 2000-06-01 ... 2022-11-30T21:00:00
    expver     int32 4B ...
Data variables:
    r          (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>
    shear      (time, latitude, longitude) float64 14GB dask.array<chunksize=(988, 165, 313), meta=np.ndarray>
    sst        (time, latitude, longitude) float64 14GB dask.array<chunksize=(737, 165, 313), meta=np.ndarray>
    sp         (time, latitude, longitude) float64 14GB dask.array<chunksize=(737, 165, 313), meta=np.ndarray>

In [16]:
final_frame.to_netcdf(f'{save_fol}/RH_SST_SHEAR_SP_2000_2023.nc')

KeyboardInterrupt: 

In [ ]:
import cartopy.crs as ccrs

fig,ax = plt.subplots(2,2,figsize=(20,10),subplot_kw={'projection':ccrs.PlateCarree()})

final_frame['shear'].isel(time=0).plot(ax=ax[0,0],transform=ccrs.PlateCarree(),)
ax[0,0].coastlines()

final_frame['sst'].isel(time=0).plot(ax=ax[0,1],transform=ccrs.PlateCarree())

ax[0,1].coastlines()

final_frame['sp'].isel(time=0).plot(ax=ax[1,0],transform=ccrs.PlateCarree())

ax[1,0].coastlines()

final_frame['r'].isel(time=0).plot(ax=ax[1,1],transform=ccrs.PlateCarree())

ax[1,1].coastlines()

In [ ]:

load_final_frame = xr.open_dataset('/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/RH_SST_SHEAR_SP_2000_2023.nc',chunks={'time':1000})


coriolis_data  = cor_data =  2 * 7.29 * 1e-5 * np.sin(load_final_frame['latitude'])

n_data = np.repeat(coriolis_data.values[:,np.newaxis],len(load_final_frame['longitude']),axis=1)

n_data_time = np.repeat(n_data[np.newaxis],len(load_final_frame['time']),axis=0)


cor_dset = xr.DataArray(data=n_data_time,name='cor_parameter',
                        dims=['time','latitude','longitude'],
                        coords=dict(
                            latitude=(["latitude"], load_final_frame.latitude.values),
                            longitude=(["longitude"], load_final_frame.longitude.values),
                            time = (["time"], load_final_frame.time.values)))

In [ ]:
load_final_frame['cor_parameter'] = cor_dset
load_final_frame['cor_parameter'].attrs['units'] = '1/s'

In [ ]:
load_final_frame.to_netcdf('/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/RH_SST_SHEAR_SP_COR_2000_2023.nc')

In [ ]:
load_final_frame['cor_parameter'].isel(time=0).plot()

In [ ]:
dset_speed.to_netcdf('/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/WIND_SPEED_2000_2023.nc')

In [ ]:
dset_all = xr.open_mfdataset('')

In [4]:
dset_all_fol = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/raw/RH_SST_SHEAR_SP_COR_2000_2023.nc'


dset_all = xr.open_mfdataset(dset_all_fol,chunks={'time':1000},parallel=True)

dset_all


<xarray.Dataset> Size: 63GB
Dimensions:        (longitude: 313, latitude: 165, time: 33672)
Coordinates:
  * longitude      (longitude) float32 1kB -83.0 -82.75 -82.5 ... -5.25 -5.0
  * latitude       (latitude) float32 660B 43.0 42.75 42.5 ... 2.5 2.25 2.0
  * time           (time) datetime64[ns] 269kB 2000-06-01 ... 2022-11-30T21:0...
    expver         int32 4B ...
Data variables:
    r              (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>
    shear          (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>
    sst            (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>
    sp             (time, latitude, longitude) float64 14GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>
    cor_parameter  (time, latitude, longitude) float32 7GB dask.array<chunksize=(1000, 165, 313), meta=np.ndarray>

In [18]:
infol = '/home1/nalex2023/main/tropcyc/atlantic_exps3/Datasets/dset_6h/'

years = np.unique(dset_all['time.year'].values.tolist())


for year in years:
    dset_year_data = dset_all.sel(time=str(year))
    dset_year_data = dset_year_data.resample(time='6h').mean()
    dset_year_data.to_netcdf(infol+f'RH_SST_SHEAR_SP_COR_6H_{year}.nc')

ai/               Climt1-70217.err  Climt1-70611.out  Climt1-70673.err
Climt1-70040.err  Climt1-70217.out  Climt1-70612.err  Climt1-70673.out
Climt1-70040.out  Climt1-70218.err  Climt1-70612.out  Climt1-70674.err
Climt1-70041.err  Climt1-70218.out  Climt1-70613.err  Climt1-70674.out
Climt1-70041.out  Climt1-70219.err  Climt1-70613.out  Climt1-70675.err
Climt1-70043.err  Climt1-70219.out  Climt1-70614.err  Climt1-70675.out
Climt1-70043.out  Climt1-70220.err  Climt1-70614.out  Climt1-70676.err
Climt1-70044.err  Climt1-70220.out  Climt1-70615.err  Climt1-70676.out
Climt1-70044.out  Climt1-70221.err  Climt1-70615.out  Climt1-70677.err
Climt1-70045.err  Climt1-70221.out  Climt1-70616.err  Climt1-70677.out
Climt1-70045.out  Climt1-70222.err  Climt1-70616.out  Climt1-70678.err
Climt1-70046.err  Climt1-70222.out  Climt1-70617.err  Climt1-70678.out
Climt1-70046.out  Climt1-70223.err  Climt1-70617.out  Climt1-70679.err
Climt1-70047.err  Climt1-70223.out  Climt1-70618.err  Climt1-70679.out
Climt1

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox

# Function to gather and format the configuration
def get_configuration():
    config = {
        "Input Shape": input_shape_var.get(),
        "Input Channels": input_channels_var.get(),
        "Conv2D Layer 1 Filters": conv1_filters_var.get(),
        "Conv2D Layer 1 Kernel Size": conv1_kernel_var.get(),
        "MaxPool2D Layer 1 Size": maxpool1_size_var.get(),
        "Conv2D Layer 2 Filters": conv2_filters_var.get(),
        "Conv2D Layer 2 Kernel Size": conv2_kernel_var.get(),
        "Dropout after Layer 2": dropout1_var.get(),
        "Conv2D Layer 3 Filters": conv3_filters_var.get(),
        "Conv2D Layer 3 Kernel Size": conv3_kernel_var.get(),
        "Conv2D Layer 4 Filters": conv4_filters_var.get(),
        "Conv2D Layer 4 Kernel Size": conv4_kernel_var.get(),
        "L2 Regularization": l2_reg_var.get(),
        "MaxPool2D Layer 2 Size": maxpool2_size_var.get(),
        "Dense Layer Units": dense1_units_var.get(),
        "Dropout after Dense": dropout2_var.get(),
        "Output Units": output_units_var.get(),
        "Activation Function": activation_var.get(),
        "Output Activation": output_activation_var.get(),
    }
    return config

# Function to show the config in a popup
def show_configuration():
    config = get_configuration()
    config_text = "\n".join([f"{k}: {v}" for k, v in config.items()])
    messagebox.showinfo("Model Configuration", config_text)

# Function to update the architecture display on the right
def update_architecture(*args):
    config = get_configuration()
    arch_lines = [
        f"Input: shape=({config['Input Shape']}, {config['Input Shape']}, {config['Input Channels']})",
        f"Conv2D(filters={config['Conv2D Layer 1 Filters']}, kernel=({config['Conv2D Layer 1 Kernel Size']}, {config['Conv2D Layer 1 Kernel Size']}), activation={config['Activation Function']})",
        f"MaxPool2D(pool_size=({config['MaxPool2D Layer 1 Size']}, {config['MaxPool2D Layer 1 Size']}))",
        f"Conv2D(filters={config['Conv2D Layer 2 Filters']}, kernel=({config['Conv2D Layer 2 Kernel Size']}, {config['Conv2D Layer 2 Kernel Size']}), activation={config['Activation Function']})",
        f"Dropout({config['Dropout after Layer 2']})",
        f"Conv2D(filters={config['Conv2D Layer 3 Filters']}, kernel=({config['Conv2D Layer 3 Kernel Size']}, {config['Conv2D Layer 3 Kernel Size']}), activation={config['Activation Function']})",
        f"Conv2D(filters={config['Conv2D Layer 4 Filters']}, kernel=({config['Conv2D Layer 4 Kernel Size']}, {config['Conv2D Layer 4 Kernel Size']}), activation={config['Activation Function']}, l2={config['L2 Regularization']})",
        f"MaxPool2D(pool_size=({config['MaxPool2D Layer 2 Size']}, {config['MaxPool2D Layer 2 Size']}))",
        f"Flatten()",
        f"Dense({config['Dense Layer Units']}, activation={config['Activation Function']})",
        f"Dropout({config['Dropout after Dense']})",
        f"Dense({config['Output Units']}, activation={config['Output Activation']})"
    ]
    arch_text = "\n".join(arch_lines)
    arch_textbox.config(state='normal')
    arch_textbox.delete(1.0, tk.END)
    arch_textbox.insert(tk.END, arch_text)
    arch_textbox.config(state='disabled')

# Create the main window
root = tk.Tk()
root.title("TensorFlow Model Configuration GUI")

# Create a frame for the parameter controls (left) and architecture (right)
frm = ttk.Frame(root, padding=12)
frm.grid(row=0, column=0, sticky="nsew")
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

# Variables for each parameter, with your defaults
input_shape_var = tk.StringVar(value='40')
input_channels_var = tk.StringVar(value='9')
conv1_filters_var = tk.StringVar(value='32')
conv1_kernel_var = tk.StringVar(value='1')
maxpool1_size_var = tk.StringVar(value='4')
conv2_filters_var = tk.StringVar(value='32')
conv2_kernel_var = tk.StringVar(value='3')
dropout1_var = tk.StringVar(value='0.25')
conv3_filters_var = tk.StringVar(value='64')
conv3_kernel_var = tk.StringVar(value='3')
conv4_filters_var = tk.StringVar(value='32')
conv4_kernel_var = tk.StringVar(value='1')
l2_reg_var = tk.StringVar(value='0.1')
maxpool2_size_var = tk.StringVar(value='2')
dense1_units_var = tk.StringVar(value='32')
dropout2_var = tk.StringVar(value='0.1')
output_units_var = tk.StringVar(value='1')
activation_var = tk.StringVar(value='relu')
output_activation_var = tk.StringVar(value='relu')

# Helper function to add a labeled entry to the GUI
def add_param(label, var, row):
    ttk.Label(frm, text=label).grid(column=0, row=row, sticky="w")
    entry = ttk.Entry(frm, textvariable=var, width=10)
    entry.grid(column=1, row=row)
    # Update architecture whenever a parameter changes
    var.trace_add('write', update_architecture)

# Add all parameters to the GUI
add_param("Input Shape", input_shape_var, 0)
add_param("Input Channels", input_channels_var, 1)
add_param("Conv2D Layer 1 Filters", conv1_filters_var, 2)
add_param("Conv2D Layer 1 Kernel Size", conv1_kernel_var, 3)
add_param("MaxPool2D Layer 1 Size", maxpool1_size_var, 4)
add_param("Conv2D Layer 2 Filters", conv2_filters_var, 5)
add_param("Conv2D Layer 2 Kernel Size", conv2_kernel_var, 6)
add_param("Dropout after Layer 2", dropout1_var, 7)
add_param("Conv2D Layer 3 Filters", conv3_filters_var, 8)
add_param("Conv2D Layer 3 Kernel Size", conv3_kernel_var, 9)
add_param("Conv2D Layer 4 Filters", conv4_filters_var, 10)
add_param("Conv2D Layer 4 Kernel Size", conv4_kernel_var, 11)
add_param("L2 Regularization", l2_reg_var, 12)
add_param("MaxPool2D Layer 2 Size", maxpool2_size_var, 13)
add_param("Dense Layer Units", dense1_units_var, 14)
add_param("Dropout after Dense", dropout2_var, 15)
add_param("Output Units", output_units_var, 16)
add_param("Activation Function", activation_var, 17)
add_param("Output Activation", output_activation_var, 18)

# Button to show the current configuration in a popup
ttk.Button(frm, text="Show Configuration", command=show_configuration).grid(column=0, row=19, columnspan=2, pady=10)

# Create a frame for the architecture display on the right
arch_frame = ttk.Frame(root, padding=12)
arch_frame.grid(row=0, column=1, sticky="nsew")
root.columnconfigure(1, weight=1)

# Label for architecture
ttk.Label(arch_frame, text="Model Architecture", font=("Arial", 12, "bold")).grid(row=0, column=0, sticky="w")

# Text widget to show the architecture; set to readonly
arch_textbox = tk.Text(arch_frame, width=40, height=20, state='disabled', font=("Courier", 10))
arch_textbox.grid(row=1, column=0, sticky="nsew")
arch_frame.rowconfigure(1, weight=1)
arch_frame.columnconfigure(0, weight=1)

# Initial architecture update
update_architecture()

# Start the Tkinter main loop
root.mainloop()

In [1]:
"""
TensorFlow Model Configuration GUI + Visualizer
-----------------------------------------------
- Left: Parameter controls (edit model architecture params)
- Right: Model architecture summary (updates live)
- "Show Configuration": shows all parameters in popup
- "Show Visual": builds a Keras model from params and visualizes using visualkeras (shows and saves output.png)
- No changes from your original model unless necessary; all layers and order preserved.
"""

import tkinter as tk
from tkinter import ttk, messagebox
import visualkeras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
import tensorflow as tf

# ---- MODEL BUILD FUNCTION ----
def build_model_from_params(params):
    """
    Build the Keras Sequential model from parameter dict.
    Structure strictly follows user's supplied model.
    """
    model = Sequential()
    regularizer = tf.keras.regularizers.l2(float(params['l2_reg']))

    # Layer 1
    model.add(Conv2D(
        filters=int(params['conv1_filters']),
        kernel_size=(int(params['conv1_kernel']), int(params['conv1_kernel'])),
        padding='Same',
        activation=params['activation'],
        input_shape=(int(params['input_shape']), int(params['input_shape']), int(params['input_channels']))
    ))
    # Layer 2
    model.add(MaxPool2D(pool_size=(int(params['maxpool1_size']), int(params['maxpool1_size']))))
    # Layer 3
    model.add(Conv2D(
        filters=int(params['conv2_filters']),
        kernel_size=(int(params['conv2_kernel']), int(params['conv2_kernel'])),
        padding='Same',
        activation=params['activation'],
    ))
    # Layer 4
    model.add(Dropout(float(params['dropout1'])))
    # Layer 5
    model.add(Conv2D(
        filters=int(params['conv3_filters']),
        kernel_size=(int(params['conv3_kernel']), int(params['conv3_kernel'])),
        padding='Same',
        activation=params['activation'],
    ))
    # Layer 6
    model.add(Conv2D(
        filters=int(params['conv4_filters']),
        kernel_size=(int(params['conv4_kernel']), int(params['conv4_kernel'])),
        padding='Same',
        activation=params['activation'],
        kernel_regularizer=regularizer
    ))
    # Layer 7
    model.add(MaxPool2D(pool_size=(int(params['maxpool2_size']), int(params['maxpool2_size']))))
    # Layer 8
    model.add(Flatten())
    # Layer 9
    model.add(Dense(int(params['dense1_units']), activation=params['activation']))
    # Layer 10
    model.add(Dropout(float(params['dropout2'])))
    # Layer 11
    model.add(Dense(int(params['output_units']), activation=params['output_activation']))
    return model

# ---- VISUALIZER FUNCTION ----
def show_model_architecture(model, filename='output.png', show=True):
    """
    Visualize given Keras model using visualkeras.layered_view.
    Saves to file and optionally displays.
    """
    img = visualkeras.layered_view(model)
    img.save(filename)
    if show:
        img.show()
    return img

# ---- ARCHITECTURE SUMMARY FUNCTION ----
def get_architecture_summary(config):
    """
    Return a summary of the architecture as text, matching the model.
    """
    arch_lines = [
        f"Input: shape=({config['input_shape']}, {config['input_shape']}, {config['input_channels']})",
        f"Conv2D(filters={config['conv1_filters']}, kernel=({config['conv1_kernel']}, {config['conv1_kernel']}), activation={config['activation']})",
        f"MaxPool2D(pool_size=({config['maxpool1_size']}, {config['maxpool1_size']}))",
        f"Conv2D(filters={config['conv2_filters']}, kernel=({config['conv2_kernel']}, {config['conv2_kernel']}), activation={config['activation']})",
        f"Dropout({config['dropout1']})",
        f"Conv2D(filters={config['conv3_filters']}, kernel=({config['conv3_kernel']}, {config['conv3_kernel']}), activation={config['activation']})",
        f"Conv2D(filters={config['conv4_filters']}, kernel=({config['conv4_kernel']}, {config['conv4_kernel']}), activation={config['activation']}, l2={config['l2_reg']})",
        f"MaxPool2D(pool_size=({config['maxpool2_size']}, {config['maxpool2_size']}))",
        f"Flatten()",
        f"Dense({config['dense1_units']}, activation={config['activation']})",
        f"Dropout({config['dropout2']})",
        f"Dense({config['output_units']}, activation={config['output_activation']})"
    ]
    return "\n".join(arch_lines)

# ---- GUI ----
root = tk.Tk()
root.title("TensorFlow Model Configurator & Visualizer")

# Main frames
frm = ttk.Frame(root, padding=12)
frm.grid(row=0, column=0, sticky="nsew")
arch_frame = ttk.Frame(root, padding=12)
arch_frame.grid(row=0, column=1, sticky="nsew")

root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=1)
root.rowconfigure(0, weight=1)

# ---- PARAMETER VARIABLES ----
input_shape_var = tk.StringVar(value='40')
input_channels_var = tk.StringVar(value='9')
conv1_filters_var = tk.StringVar(value='32')
conv1_kernel_var = tk.StringVar(value='1')
maxpool1_size_var = tk.StringVar(value='4')
conv2_filters_var = tk.StringVar(value='32')
conv2_kernel_var = tk.StringVar(value='3')
dropout1_var = tk.StringVar(value='0.25')
conv3_filters_var = tk.StringVar(value='64')
conv3_kernel_var = tk.StringVar(value='3')
conv4_filters_var = tk.StringVar(value='32')
conv4_kernel_var = tk.StringVar(value='1')
l2_reg_var = tk.StringVar(value='0.1')
maxpool2_size_var = tk.StringVar(value='2')
dense1_units_var = tk.StringVar(value='32')
dropout2_var = tk.StringVar(value='0.1')
output_units_var = tk.StringVar(value='1')
activation_var = tk.StringVar(value='relu')
output_activation_var = tk.StringVar(value='relu')

# ---- HELPER FUNCTION FOR PARAM ENTRIES ----
def add_param(label, var, row):
    ttk.Label(frm, text=label).grid(column=0, row=row, sticky="w")
    entry = ttk.Entry(frm, textvariable=var, width=10)
    entry.grid(column=1, row=row)
    var.trace_add('write', update_architecture)

# ---- ALL PARAMS ----
add_param("Input Shape", input_shape_var, 0)
add_param("Input Channels", input_channels_var, 1)
add_param("Conv2D Layer 1 Filters", conv1_filters_var, 2)
add_param("Conv2D Layer 1 Kernel Size", conv1_kernel_var, 3)
add_param("MaxPool2D Layer 1 Size", maxpool1_size_var, 4)
add_param("Conv2D Layer 2 Filters", conv2_filters_var, 5)
add_param("Conv2D Layer 2 Kernel Size", conv2_kernel_var, 6)
add_param("Dropout after Layer 2", dropout1_var, 7)
add_param("Conv2D Layer 3 Filters", conv3_filters_var, 8)
add_param("Conv2D Layer 3 Kernel Size", conv3_kernel_var, 9)
add_param("Conv2D Layer 4 Filters", conv4_filters_var, 10)
add_param("Conv2D Layer 4 Kernel Size", conv4_kernel_var, 11)
add_param("L2 Regularization", l2_reg_var, 12)
add_param("MaxPool2D Layer 2 Size", maxpool2_size_var, 13)
add_param("Dense Layer Units", dense1_units_var, 14)
add_param("Dropout after Dense", dropout2_var, 15)
add_param("Output Units", output_units_var, 16)
add_param("Activation Function", activation_var, 17)
add_param("Output Activation", output_activation_var, 18)

# ---- GATHER PARAMS ----
def get_config():
    return {
        "input_shape": input_shape_var.get(),
        "input_channels": input_channels_var.get(),
        "conv1_filters": conv1_filters_var.get(),
        "conv1_kernel": conv1_kernel_var.get(),
        "maxpool1_size": maxpool1_size_var.get(),
        "conv2_filters": conv2_filters_var.get(),
        "conv2_kernel": conv2_kernel_var.get(),
        "dropout1": dropout1_var.get(),
        "conv3_filters": conv3_filters_var.get(),
        "conv3_kernel": conv3_kernel_var.get(),
        "conv4_filters": conv4_filters_var.get(),
        "conv4_kernel": conv4_kernel_var.get(),
        "l2_reg": l2_reg_var.get(),
        "maxpool2_size": maxpool2_size_var.get(),
        "dense1_units": dense1_units_var.get(),
        "dropout2": dropout2_var.get(),
        "output_units": output_units_var.get(),
        "activation": activation_var.get(),
        "output_activation": output_activation_var.get(),
    }

# ---- SHOW CONFIG BUTTON ----
def show_configuration():
    config = get_config()
    config_text = "\n".join([f"{k}: {v}" for k, v in config.items()])
    messagebox.showinfo("Model Configuration", config_text)

ttk.Button(frm, text="Show Configuration", command=show_configuration).grid(column=0, row=19, columnspan=2, pady=10)

# ---- SHOW VISUAL BUTTON ----
def show_visual():
    try:
        params = get_config()
        # Build the actual model from parameters
        model = build_model_from_params(params)
        # Visualize and show/save
        show_model_architecture(model, filename='output.png', show=True)
        messagebox.showinfo("Model Visualizer", "Model visual saved to output.png and displayed.")
    except Exception as e:
        messagebox.showerror("Error", f"Could not build/visualize model: {e}")

ttk.Button(frm, text="Show Visual", command=show_visual).grid(column=0, row=20, columnspan=2, pady=4)

# ---- ARCHITECTURE DISPLAY ----
ttk.Label(arch_frame, text="Model Architecture", font=("Arial", 12, "bold")).grid(row=0, column=0, sticky="w")
arch_textbox = tk.Text(arch_frame, width=40, height=20, state='disabled', font=("Courier", 10))
arch_textbox.grid(row=1, column=0, sticky="nsew")
arch_frame.rowconfigure(1, weight=1)
arch_frame.columnconfigure(0, weight=1)

def update_architecture(*args):
    config = get_config()
    arch_text = get_architecture_summary(config)
    arch_textbox.config(state='normal')
    arch_textbox.delete(1.0, tk.END)
    arch_textbox.insert(tk.END, arch_text)
    arch_textbox.config(state='disabled')

update_architecture()

root.mainloop()

NameError: name 'update_architecture' is not defined